# Authorization

https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models


https://github.com/GoogleCloudPlatform/python-docs-samples/tree/ecd48fab644676be7bcf86d2ffd8ead58b8f0e3f/generative_ai


# Installing the package

In [ ]:
!pip install --quiet google-cloud-aiplatform --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
#!gsutil cp gs://vertex_sdk_llm_private_releases/SDK/google_cloud_aiplatform-1.23.0.llm.alpha.23.03.28-py2.py3-none-any.whl .
#!pip install invoke
#!pip install google_cloud_aiplatform-1.23.0.llm.alpha.23.03.28-py2.py3-none-any.whl "shapely<2.0.0"

# ! ^^^^ Do not forget to click the "Restart runtime" button above.

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
PROJECT_ID = "<provide a project id>" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [ ]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from vertexai.preview.language_models import TextGenerationModel, TextEmbeddingModel, CodeGenerationModel
model = TextGenerationModel.from_pretrained("text-bison@001")

In [ ]:

def predict_large_language_model_sample(
    prompt: str

    ) :
    """Predict using a Large Language Model."""
    example = ''' Extract all information as JSON:
                  input: calculate interest and total amount paid on a Loan Principal of $32,000 over 10 years, with an interest rate of 5 % per annum.

                  This quote is being prepared for the following customer details
                  customer: John Doe
                  Address: 52 west street, Columbus, Ohio, 43215
                  Phone: 543-201-0100

                  output: {"customer": "John Doe",
                  "Address": "52 west street",
                  "City": "Columbus"
                  "State": "Ohio"
                  "Zip":  "43215",
                  "Phone": "543-201-0100",
                  "Loan Amount": "$32,000",
                  "Term": "10 years"
                  "Interest Rate": "5%"
                  "Total Interest": "$16000",
                  "Total Amount to be paid": "$48000"
                  }

                  '''

    response = model.predict(
        example + prompt,
        temperature=0,
        max_output_tokens=1000,
        top_k=1,
        top_p=0,)
    #print(f"Response from Model: {response.text}")
    return response.text


#For testing of the above function
json_resp = predict_large_language_model_sample('''
input: calculate interest and total amount paid on a Loan Principal of $10,000 over 10 years, with an interest rate of 5 % per annum.

This quote is being prepared for the following customer details
customer: John Doe
Address: 52 west street, Columbus, Ohio, 43215
Phone: 543-201-0100

output:
''')

print("json format \n", json_resp)

json format 
 {"customer": "John Doe",
  "Address": "52 west street",
  "City": "Columbus"
  "State": "Ohio"
  "Zip":  "43215",
  "Phone": "543-201-0100",
  "Loan Amount": "$10,000",
  "Term": "10 years"
  "Interest Rate": "5%"
  "Total Interest": "$5000",
  "Total Amount to be paid": "$15000"
  }


In [ ]:
code_model = CodeGenerationModel.from_pretrained("code-bison@001")

def generate_code(var_json_resp):

  instructions = "convert the following json in an insert statement that can be inserted a in QUOTES table in bigquery. "
  code_prompt = instructions + " " +var_json_resp
  #print(code_prompt)

  response = code_model .predict(
          code_prompt,
          temperature=0.2,
          max_output_tokens=1500)
  #print(f"Response from Model: {response.text}")
  return response.text


#for testing of above statement
insert_statement = generate_code(json_resp)
print(insert_statement)

```
INSERT INTO QUOTES (
  customer,
  address,
  city,
  state,
  zip,
  phone,
  loan_amount,
  term,
  interest_rate,
  total_interest,
  total_amount_to_be_paid
)
VALUES (
  'John Doe',
  '52 west street',
  'Columbus',
  'Ohio',
  '43215',
  '543-201-0100',
  '$10,000',
  '10 years',
  '5%',
  '$5000',
  '$15000'
);
```


In [ ]:
from google.cloud import bigquery as bq

def bq_insert(bq_insert_statement):

  # Set the project ID and dataset ID
  project_id = "mss-project1"
  dataset_id = "genai_dataset"

  # Create a BigQuery client
  client = bq.Client(project_id)

  # Create a table
  table = client.dataset(dataset_id).table("quotes")

  # Insert the row
  row = {
    'customer': 'John Doe',
    'address': '52 west street',
    'city': 'Columbus',
    'state': 'Ohio',
    'zip': '43215',
    'phone': '543-201-0100',
    'loan_amount': '$10,000',
    'term': '36',
    'interest_rate': '5.0',
    'total_interest': '1500.0',
    'total_amount_to_be_paid': '11500.0'
  }
  row = [
  "John Doe",
  "52 west street",
  "Columbus",
  "Ohio",
  "43215",
  "543-201-0100",
  "10000",
  "10 years",
  "5",
  "5000",
  "15000"
]

  # Insert the row into the table
  client.insert_rows(table, [row], selected_fields=["customer", "address", "city", "state", "zip", "phone", "loan_amount", "term", "interest_rate", "total_interest", "total_amount_to_be_paid"])
  status_response= "Row Inserted into BQ"
  return status_response


In [ ]:
print(bq_insert(""))

AttributeError: ignored

In [ ]:
!pip install --quiet gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:0

Use this :

```
calculate interest and total amount paid on a Loan Principal of $10,000 over 10 years, with an interest rate of 5 % per annum.

This quote is being prepared for the following customer details
customer: John Doe
Address: 52 west street, Columbus, Ohio, 43215
Phone: 543-201-0100
 ```



In [ ]:
import gradio as gr

def qa_interface(question):
    QUESTION = question
    PROMPT = "Extract all information as JSON. input: " + QUESTION + "output:"

    json_op = predict_large_language_model_sample(PROMPT)
    bq_insert_statement = generate_code(json_op)
    #bq_status = bq_insert(bq_insert_statement)
    bq_status = "Trying...."
    return bq_status, json_op, bq_insert_statement

iface = gr.Interface(fn=qa_interface,
                     inputs=[
                         gr.inputs.Textbox(label="Question")
                         #, gr.Slider(0, 1, 0.3)
                         #, gr.Slider(0, 1, 1)
                         #, gr.Slider(0, 1024, 700)
                         #, gr.Slider(0, 40, 40)
                         ],
                     outputs=[
                         gr.Textbox(label ="Action"),
                         gr.Textbox(label="JSON Output"),
                         gr.Textbox(label="BigQuery Insert Statement")

                          ],
                     title="Loan Quotes Generator",
                     description="This is a GenAI based quotes generator (using Google Gen APIs)",
                     allow_flagging=False,
                     theme=gr.themes.Soft()
                     )

iface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:325: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d67f0f5b871a8d5495.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
